In [117]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from streamlit_folium import folium_static
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
import plotly.graph_objects as go
import plotly.express as px


In [118]:
# Read data
df = pd.read_csv("Lengkap-Dataset_Longsor 2021 - 2023 - PROV.csv")

In [119]:
df.describe()

,No,TAHUN,JUMLAH_LONGSOR,JIWA_TERDAMPAK,JIWA_MENINGGAL,RUSAK_TERDAMPAK,RUSAK_RINGAN,RUSAK_SEDANG,RUSAK_BERAT,TERTIMBUN,LATITUDE,LONGITUDE
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,41.000000,2022.000000,25.814815,234.802469,1.197531,56.271605,23.851852,21.012346,11.567901,1.765432,-6.833017,107.686291
std,23.526581,0.821584,39.355149,464.974473,2.170367,98.455194,42.522380,40.683686,21.402651,7.381178,0.368131,0.630241
min,1.000000,2021.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7.700090,106.550721
25%,21.000000,2021.000000,2.000000,5.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,-7.021893,107.139363
50%,41.000000,2022.000000,9.000000,56.000000,0.000000,17.000000,6.000000,6.000000,3.000000,0.000000,-6.835020,107.609837
75%,61.000000,2023.000000,32.000000,285.000000,2.000000,67.000000,27.000000,20.000000,11.000000,0.000000,-6.557067,108.323123
max,81.000000,2023.000000,225.000000,2673.000000,12.000000,476.000000,213.000000,220.000000,132.000000,61.000000,-6.235769,108.563810


In [120]:
df.head()

,No,KABUPATEN,TAHUN,JUMLAH_LONGSOR,JIWA_TERDAMPAK,JIWA_MENINGGAL,RUSAK_TERDAMPAK,RUSAK_RINGAN,RUSAK_SEDANG,RUSAK_BERAT,TERTIMBUN,LATITUDE,LONGITUDE
0,1,KABUPATEN BOGOR,2021,225,2673,0,405,213,143,49,0,-6.479036,106.824545
1,2,KABUPATEN SUKABUMI,2021,147,311,3,82,24,39,19,14,-6.988837,106.550721
2,3,KABUPATEN CIANJUR,2021,27,227,1,48,14,24,10,0,-6.822804,107.139363
3,4,KABUPATEN BANDUNG,2021,60,285,3,44,17,13,14,15,-7.021893,107.527727
4,5,KABUPATEN GARUT,2021,42,294,0,17,1,6,10,2,-7.202182,107.887006


In [121]:
# Choose the column for the line chart
selected_column = 'JUMLAH_LONGSOR'

In [122]:
# Calculate quartiles
quartiles = df[selected_column].quantile([0.25, 0.5, 0.75])

In [123]:
# Create line chart for each quartile
fig_q1 = px.line(df, x=df.index, y=selected_column, title="Line Chart - 25th Percentile (Q1)")
fig_q1.update_layout(height=300, width=400)  # Adjust the size
fig_q1.show()



# Print quartile information
print(f"- 0.25% Percentile (Q1): {quartiles[0.25]}")


- 0.25% Percentile (Q1): 2.0


In [124]:
fig_q2 = px.line(df, x=df.index, y=selected_column, title="Line Chart - 50th Percentile (Q2)")
fig_q2.update_layout(height=300, width=400)  # Adjust the size
fig_q2.show()

print(f"- 50th Percentile (Q2): {quartiles[0.5]}")


- 50th Percentile (Q2): 9.0


In [125]:

fig_q3 = px.line(df, x=df.index, y=selected_column, title="Line Chart - 75th Percentile (Q3)")
fig_q3.update_layout(height=300, width=400)  # Adjust the size
fig_q3.show()

print(f"- 75th Percentile (Q3): {quartiles[0.75]}")


- 75th Percentile (Q3): 32.0


In [126]:
# Exploring variables
selected_features = [
    'JUMLAH_LONGSOR',
    'JIWA_TERDAMPAK',
    'JIWA_MENINGGAL',
    'RUSAK_TERDAMPAK',
    'RUSAK_RINGAN',
    'RUSAK_SEDANG',
    'RUSAK_BERAT',
    'TERTIMBUN',
    'LATITUDE',
    'LONGITUDE'
]



In [127]:
# Calculate correlation matrix
correlation_matrix = df[selected_features].corr()

# Plot heatmap using Plotly Express
fig = px.imshow(correlation_matrix,
                labels=dict(x="Features", y="Features", color="Correlation"),
                x=selected_features,
                y=selected_features,
                color_continuous_scale="viridis",
                title="Heatmap Korelasi")

fig.show()

print("Visualisasi ini memberikan gambaran distribusi univariat dari setiap variabel dalam dataset. Histogram menunjukkan sebaran nilai-nilai di setiap variabel, dan kernel density estimation (KDE) memberikan perkiraan kurva distribusi.")


Visualisasi ini memberikan gambaran distribusi univariat dari setiap variabel dalam dataset. Histogram menunjukkan sebaran nilai-nilai di setiap variabel, dan kernel density estimation (KDE) memberikan perkiraan kurva distribusi.


In [128]:
df.isnull().sum()


No                 0
KABUPATEN          0
TAHUN              0
JUMLAH_LONGSOR     0
JIWA_TERDAMPAK     0
JIWA_MENINGGAL     0
RUSAK_TERDAMPAK    0
RUSAK_RINGAN       0
RUSAK_SEDANG       0
RUSAK_BERAT        0
TERTIMBUN          0
LATITUDE           0
LONGITUDE          0
dtype: int64

In [129]:

df.describe().T

,count,mean,std,min,25%,50%,75%,max
No,81.0,41.000000,23.526581,1.000000,21.000000,41.000000,61.000000,81.000000
TAHUN,81.0,2022.000000,0.821584,2021.000000,2021.000000,2022.000000,2023.000000,2023.000000
JUMLAH_LONGSOR,81.0,25.814815,39.355149,0.000000,2.000000,9.000000,32.000000,225.000000
JIWA_TERDAMPAK,81.0,234.802469,464.974473,0.000000,5.000000,56.000000,285.000000,2673.000000
JIWA_MENINGGAL,81.0,1.197531,2.170367,0.000000,0.000000,0.000000,2.000000,12.000000
RUSAK_TERDAMPAK,81.0,56.271605,98.455194,0.000000,2.000000,17.000000,67.000000,476.000000
RUSAK_RINGAN,81.0,23.851852,42.522380,0.000000,0.000000,6.000000,27.000000,213.000000
RUSAK_SEDANG,81.0,21.012346,40.683686,0.000000,1.000000,6.000000,20.000000,220.000000
RUSAK_BERAT,81.0,11.567901,21.402651,0.000000,0.000000,3.000000,11.000000,132.000000
TERTIMBUN,81.0,1.765432,7.381178,0.000000,0.000000,0.000000,0.000000,61.000000


In [130]:
# Select numerical columns for clustering
numeric_columns = ['JUMLAH_LONGSOR', 'JIWA_TERDAMPAK', 'JIWA_MENINGGAL', 'RUSAK_TERDAMPAK', 'RUSAK_RINGAN', 'RUSAK_SEDANG', 'RUSAK_BERAT', 'TERTIMBUN', 'LATITUDE', 'LONGITUDE']
X_ahc = df[['JUMLAH_LONGSOR', 'JIWA_TERDAMPAK', 'JIWA_MENINGGAL', 'RUSAK_TERDAMPAK', 'RUSAK_RINGAN', 'RUSAK_SEDANG', 'RUSAK_BERAT', 'TERTIMBUN']]

In [131]:
X_ahc

,JUMLAH_LONGSOR,JIWA_TERDAMPAK,JIWA_MENINGGAL,RUSAK_TERDAMPAK,RUSAK_RINGAN,RUSAK_SEDANG,RUSAK_BERAT,TERTIMBUN
0,225,2673,0,405,213,143,49,0
1,147,311,3,82,24,39,19,14
2,27,227,1,48,14,24,10,0
3,60,285,3,44,17,13,14,15
4,42,294,0,17,1,6,10,2
...,...,...,...,...,...,...,...,...
76,1,0,0,8,6,2,0,0
77,2,0,0,3,0,0,3,0
78,11,34,12,1,10,1,0,0
79,10,103,0,40,17,14,9,0


In [132]:
# Single Linkage
linkage_matrix_ward = linkage(X_ahc, method='single')
cophenet_matrix_ward, _ = cophenet(linkage_matrix_ward, pdist(X_ahc))
ccc_ward = cophenet_matrix_ward.mean()


In [133]:
# Complete Linkage
linkage_matrix_complete = linkage(X_ahc, method='complete')
cophenet_matrix_complete, _ = cophenet(linkage_matrix_complete, pdist(X_ahc))
ccc_complete = cophenet_matrix_complete.mean()



In [134]:
# Average Linkage
linkage_matrix_average = linkage(X_ahc, method='average')
cophenet_matrix_average, _ = cophenet(linkage_matrix_average, pdist(X_ahc))
ccc_average = cophenet_matrix_average.mean()

In [135]:
cut_height_ward = 175000
cut_height_complete = 10.0
cut_height_average = 5.0


In [136]:
labels_ward = fcluster(linkage_matrix_ward, t=cut_height_ward, criterion='distance')
labels_complete = fcluster(linkage_matrix_complete, t=cut_height_complete, criterion='distance')
labels_average = fcluster(linkage_matrix_average, t=cut_height_average, criterion='distance')


In [137]:
# Create dendrogram figures for Ward linkage
fig_ward = ff.create_dendrogram(X_ahc, orientation='bottom', linkagefun=lambda x: linkage_matrix_ward)
fig_ward.update_layout(title='Dendrogram AHC (Ward)')
fig_ward.show()

In [138]:
# Menampilkan kesimpulan
print("Cophenetic Correlation Coefficient (CCC) untuk Dendrogram AHC (Ward): {:.4f}".format(ccc_ward))
print("Jumlah klaster optimal untuk metode linkage Ward: {}".format(len(set(labels_ward))))


Cophenetic Correlation Coefficient (CCC) untuk Dendrogram AHC (Ward): 0.9465
Jumlah klaster optimal untuk metode linkage Ward: 1


In [139]:
# Create dendrogram figures for Complete linkage
fig_complete = ff.create_dendrogram(X_ahc, orientation='bottom', linkagefun=lambda x: linkage_matrix_complete)
fig_complete.update_layout(title='Dendrogram AHC (Complete)')
fig_complete.show()

In [140]:
print("Cophenetic Correlation Coefficient (CCC) untuk Dendrogram AHC (Complete): {:.4f}".format(ccc_complete))
print("Jumlah klaster optimal untuk metode linkage Complete: {}".format(len(set(labels_complete))))


Cophenetic Correlation Coefficient (CCC) untuk Dendrogram AHC (Complete): 0.9568
Jumlah klaster optimal untuk metode linkage Complete: 50


In [141]:
# Create dendrogram figures for Average linkage
fig_average = ff.create_dendrogram(X_ahc, orientation='bottom', linkagefun=lambda x: linkage_matrix_average)
fig_average.update_layout(title='Dendrogram AHC (Average)')
fig_average.show()


In [142]:
print("Cophenetic Correlation Coefficient (CCC) untuk Dendrogram AHC (Average): {:.4f}".format(ccc_average))
print("Jumlah klaster optimal untuk metode linkage Average: {}".format(len(set(labels_average))))

Cophenetic Correlation Coefficient (CCC) untuk Dendrogram AHC (Average): 0.9574
Jumlah klaster optimal untuk metode linkage Average: 57
